In [100]:
from keras.objectives import mean_squared_error

In [101]:
import tensorflow

In [102]:
import tensorflow as tf

In [21]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())

In [22]:
from configs import configs

importing Jupyter notebook from configs.ipynb


In [161]:
def mask_out_nonobjs(inp_tens, obj_tens):
    #make sure channel is last dim
    mask = tf.cast(obj_tens, dtype=tf.bool)
    masked_data = tf.boolean_mask(tensor=inp_tens, mask=mask,)
    return masked_data
    

In [162]:
def class_loss(y_pred, y_true, obj_tens):
#     y_pred = mask_out_nonobjs(y_pred, obj_tens)
#     y_true = mask_out_nonobjs(y_true, obj_tens)
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_pred,labels=y_true)
    tf.mul(losses, tf.squeeze(obj_tens))
    return tf.reduce_mean(losses)
    
    

In [163]:
def obj_loss(y_pred, y_true):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_pred,labels=y_true)
    return tf.reduce_mean(losses)
    

In [164]:
def coord_loss(y_pred, y_true, obj_tens):
    #y_pred, y_true = mask_out_nonobjs(y_pred,obj_tens), mask_out_nonobjs(y_true, obj_tens)
    losses = smooth_l1(tf.subtract(y_true, y_pred))
    tf.mul(losses, obj_tens)
    return tf.reduce_mean(losses)

In [165]:
def reconstruction_loss(y_pred, y_true):
    return tf.reduce_mean(mean_squared_error(y_pred, y_true))

In [166]:
 def smooth_l1(x):
    abs_x = tf.abs(x)
    is_less_than_1 = tf.stop_gradient(tf.to_float(tf.less(abs_x,1.)))
    
    loss = tf.pow(x, 2) * 0.5 * is_less_than_1 \
                    + (abs_x - 0.5) * (1.0 - is_less_than_1)

    return loss

In [171]:
def yolo_box_loss(y_true, y_pred):
    alpha, beta = [configs[k] for k in ["alpha", "beta"]]
    
    y_pred_xy,y_pred_wh,y_pred_obj, y_pred_cls1,y_pred_cl2 = tf.split(value=y_pred,num_split=5,split_dim=3)
    y_pred_cls = tf.concat(values=[y_pred_cls1,y_pred_cl2], concat_dim=3)
    
    y_true_xy,y_true_wh,y_true_obj_y_true_cls = tf.split(value=y_true,num_split=3,split_dim=3)
    y_true_obj, y_true_cls = tf.split(value=y_true_obj_y_true_cls, num_split=2, split_dim=3)
    
    y_mask_obj = tf.cast(y_true_obj, tf.float32)
    y_true_obj = tf.squeeze(tf.cast(y_true_obj, tf.int32))
    y_true_cls = tf.squeeze(tf.cast(y_true_cls, tf.int32))
    cls_loss = class_loss(y_pred_cls, y_true_cls,y_mask_obj)
    o_loss = obj_loss(y_pred_obj,y_true_obj)
    xy_loss = coord_loss(y_pred_xy, y_true_xy, y_mask_obj)
    wh_loss = coord_loss(y_pred_wh, y_true_wh, y_mask_obj)
    
    
    
    xy_loss = tf.scalar_mul(alpha, xy_loss)
    wh_loss = tf.scalar_mul(beta, wh_loss)
    
    loss = tf.add_n([xy_loss, wh_loss, o_loss, cls_loss])
    return loss
    

In [172]:
# def yolo_semisupervised_loss(y_true, y_pred):
#     y_reconstruction = y_pred
#     y_true_im = y_true
#     loss_rec = reconstruction_loss(y_reconstruction, y_true_im)
#     loss_box = yolo_supervised_loss(outputs, gr_truth)
#     loss = tf.add(loss_box,tf.scalar_mul(configs["lambda"], loss_box) )
#     return loss
    
    

In [173]:
if __name__ == "__main__":
    sess = tf.InteractiveSession()
    brn=tf.contrib.distributions.Bernoulli(p=0.5)
    uni = tf.contrib.distributions.Uniform(0.,1.)
    cat = tf.contrib.distributions.Categorical(p=4*[0.25])
    nrm = tf.contrib.distributions.Normal(mu=0.,sigma=1.)
    
    
    y_true_obj = tf.cast(brn.sample(sample_shape=(5,24,24,1)),tf.float32)
    

    y_pred_cls = uni.sample(sample_shape=(5,24,24, 4))
    y_pred_obj = uni.sample(sample_shape=(5,24,24, 2))
    y_pred_xy = nrm.sample(sample_shape=(5,24,24,2))
    y_pred_wh = nrm.sample(sample_shape=(5,24,24,2))
    y_true_xy = nrm.sample(sample_shape=(5,24,24,2))
    y_true_wh = nrm.sample(sample_shape=(5,24,24,2))
    y_true_cls = tf.cast(cat.sample(sample_shape=(5,24,24,1)), tf.float32)
    
    y_true_im = nrm.sample(sample_shape=(768,768,16))
    y_pred_im = nrm.sample(sample_shape=(768,768,16))
    
    
    outputs = tf.concat(values=[y_pred_xy,y_pred_wh,y_pred_obj, y_pred_cls], concat_dim=3)
    gr_truth = tf.concat(values=[y_true_xy,y_true_wh,y_true_obj, y_true_cls], concat_dim=3)
    loss= sess.run(yolo_box_loss(gr_truth, outputs))
    #print sess.run(yolo_semisupervised_loss(outputs, gr_truth))


(5, 24, 24)
